# Build a RAG solution using Azure AI Search

Steps in this notebook:
1. Create an index 
2. Create a data source
3. Create a skillset
4. Create an indexer 
5. Send a query to the search engine to check results
6. Send query results to a language model to generate response

Note: Steps 1-4: Done during initial setup of Search Index only

## Install dependencies

In [ ]:
%pip install python-dotenv
%pip install azure-core
%pip install azure-search-documents
%pip install azure-storage-blob
%pip install azure-identity
%pip install openai
%pip install aiohttp
%pip install ipywidgets
%pip install ipykernel

## Load Azure configurations

You always need to run this!

In [2]:
from dotenv import load_dotenv
import os

load_dotenv() # take environment variables from .env.

azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
azure_openai_embeddings_deployment = os.getenv("AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT")
azure_openai_api_version = "2024-10-01-preview"
azure_openai_embedding_size = 1536
azure_search_service_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
azure_search_service_admin_key = os.getenv("AZURE_SEARCH_ADMIN_KEY")
azure_search_service_index_name = "ai-search-index-001"
azure_storage_connection_string = os.getenv("AZURE_STORAGE_CONNECTION_STRING")
azure_ai_services_key = os.getenv("AZURE_AI_MULTISERVICE_KEY")

## Create an index

In [2]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SearchField,
    SearchFieldDataType,
    VectorSearch,
    HnswAlgorithmConfiguration,
    VectorSearchProfile,
    AzureOpenAIVectorizer,
    AzureOpenAIVectorizerParameters,
    SearchIndex,
    SemanticConfiguration,
    SemanticPrioritizedFields,
    SemanticField,
    SemanticSearch,
    ScoringProfile,
    TagScoringFunction,
    TagScoringParameters
)

# Get credential from Azure AI Search Admin key
credential = AzureKeyCredential(azure_search_service_admin_key)

# Search index name  
index_name = azure_search_service_index_name

# Create a Search Index Client
index_client = SearchIndexClient(endpoint=azure_search_service_endpoint, credential=credential)

# Define the fields collection
fields = [
    SearchField(name="parent_id", type=SearchFieldDataType.String),  
    SearchField(name="title", type=SearchFieldDataType.String),
    SearchField(name="locations", type=SearchFieldDataType.Collection(SearchFieldDataType.String), filterable=True),
    SearchField(name="chunk_id", type=SearchFieldDataType.String, key=True, sortable=True, filterable=True, facetable=True, analyzer_name="keyword"),  
    SearchField(name="chunk", type=SearchFieldDataType.String, sortable=False, filterable=False, facetable=False),  
    SearchField(name="text_vector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single), vector_search_dimensions=azure_openai_embedding_size, vector_search_profile_name="myHnswProfile")
    ]  
  
# Configure the vector search configuration  
vector_search = VectorSearch(  
    algorithms=[  
        HnswAlgorithmConfiguration(name="myHnsw"),
    ],  
    profiles=[  
        VectorSearchProfile(  
            name="myHnswProfile",  
            algorithm_configuration_name="myHnsw",  
            vectorizer_name="myOpenAI",  
        )
    ],  
    vectorizers=[   # a vectorizer is software that performs vectorization
        AzureOpenAIVectorizer(  
            vectorizer_name="myOpenAI",  
            kind="azureOpenAI",  
            parameters=AzureOpenAIVectorizerParameters(  
                resource_url=azure_openai_endpoint,  
                deployment_name=azure_openai_embeddings_deployment,
                model_name=azure_openai_embeddings_deployment
            ),
        ),  
    ], 
)  

# New semantic configuration
semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=SemanticPrioritizedFields(
        title_field=SemanticField(field_name="title"),
        keywords_fields=[SemanticField(field_name="locations")],
        content_fields=[SemanticField(field_name="chunk")]
    )
)

# Create the semantic settings with the configuration
semantic_search = SemanticSearch(configurations=[semantic_config])

# New scoring profile
scoring_profiles = [  
    ScoringProfile(  
        name="my-scoring-profile",
        functions=[
            TagScoringFunction(  
                field_name="locations",  
                boost=5.0,  
                parameters=TagScoringParameters(  
                    tags_parameter="tags",  
                ),  
            ) 
        ]
    )
]

# Create the search index
index = SearchIndex(name=index_name, 
                    fields=fields, 
                    vector_search=vector_search,
                    semantic_search=semantic_search,
                    scoring_profiles=scoring_profiles)  
result = index_client.create_or_update_index(index)  
print(f"{result.name} created")

ai-search-index-001 created


## Check Azure Storage access
Verify access to storage and print out the documents

In [3]:
from azure.storage.blob import BlobServiceClient

# Initialize the BlobServiceClient with the connection string
blob_service_client = BlobServiceClient.from_connection_string(azure_storage_connection_string)

# Get the container client
container_client = blob_service_client.get_container_client("nasabooks")

# List blobs in the container
try:
    blobs_list = container_client.list_blobs()
    print("Blobs in the container:")
    for blob in blobs_list:
        print(blob.name)
    print("Access to the blob storage was granted.")
except Exception as e:
    print(f"Failed to access the blob storage: {e}")

Blobs in the container:
page-11.pdf
page-13.pdf
page-15.pdf
page-17.pdf
page-19.pdf
page-21.pdf
page-23.pdf
page-25.pdf
page-27.pdf
page-31.pdf
page-33.pdf
page-35.pdf
page-39.pdf
page-41.pdf
page-43.pdf
page-45.pdf
page-49.pdf
page-51.pdf
page-55.pdf
page-57.pdf
page-59.pdf
page-61.pdf
page-63.pdf
page-65.pdf
page-67.pdf
page-69.pdf
page-7.pdf
page-71.pdf
page-8.pdf
page-9.pdf
Access to the blob storage was granted.


## Create a Data Source

In [4]:
from azure.search.documents.indexes import SearchIndexerClient
from azure.search.documents.indexes.models import (
    SearchIndexerDataContainer,
    SearchIndexerDataSourceConnection
)

# Create a data source 
indexer_client = SearchIndexerClient(endpoint=azure_search_service_endpoint, credential=credential)
container = SearchIndexerDataContainer(name="nasabooks")
data_source_connection = SearchIndexerDataSourceConnection(
    name="ai-search-ds",
    type="azureblob",
    connection_string=azure_storage_connection_string,
    container=container
)
data_source = indexer_client.create_or_update_data_source_connection(data_source_connection)

print(f"Data source '{data_source.name}' created or updated")

Data source 'ai-search-ds' created or updated


## Create a Skillset

In [5]:
from azure.search.documents.indexes.models import (
    SplitSkill,
    InputFieldMappingEntry,
    OutputFieldMappingEntry,
    AzureOpenAIEmbeddingSkill,
    EntityRecognitionSkill,
    SearchIndexerIndexProjection,
    SearchIndexerIndexProjectionSelector,
    SearchIndexerIndexProjectionsParameters,
    IndexProjectionMode,
    SearchIndexerSkillset,
    CognitiveServicesAccountKey
)

# Create a skillset  
skillset_name = "ai-search-ss"

split_skill = SplitSkill(  
    description="Split skill to chunk documents",  
    text_split_mode="pages",  
    context="/document",  
    maximum_page_length=2000,  
    page_overlap_length=500,  
    inputs=[  
        InputFieldMappingEntry(name="text", source="/document/content"),  
    ],  
    outputs=[  
        OutputFieldMappingEntry(name="textItems", target_name="pages")  
    ],  
)  
  
embedding_skill = AzureOpenAIEmbeddingSkill(  
    description="Skill to generate embeddings via Azure OpenAI",  
    context="/document/pages/*",  
    resource_url=azure_openai_endpoint,  
    deployment_name=azure_openai_embeddings_deployment,  
    model_name=azure_openai_embeddings_deployment,
    dimensions=azure_openai_embedding_size,
    inputs=[  
        InputFieldMappingEntry(name="text", source="/document/pages/*"),  
    ],  
    outputs=[  
        OutputFieldMappingEntry(name="embedding", target_name="text_vector")  
    ],  
)

entity_skill = EntityRecognitionSkill(
    description="Skill to recognize entities in text",
    context="/document/pages/*",
    categories=["Location"],
    default_language_code="en",
    inputs=[
        InputFieldMappingEntry(name="text", source="/document/pages/*")
    ],
    outputs=[
        OutputFieldMappingEntry(name="locations", target_name="locations")
    ]
)
  
index_projections = SearchIndexerIndexProjection(  
    selectors=[  
        SearchIndexerIndexProjectionSelector(  
            target_index_name=azure_search_service_index_name,  
            parent_key_field_name="parent_id",  
            source_context="/document/pages/*",  
            mappings=[  
                InputFieldMappingEntry(name="chunk", source="/document/pages/*"),  
                InputFieldMappingEntry(name="text_vector", source="/document/pages/*/text_vector"),
                InputFieldMappingEntry(name="locations", source="/document/pages/*/locations"),  
                InputFieldMappingEntry(name="title", source="/document/metadata_storage_name"),  
            ],  
        ),  
    ],  
    parameters=SearchIndexerIndexProjectionsParameters(  
        projection_mode=IndexProjectionMode.SKIP_INDEXING_PARENT_DOCUMENTS  
    ),  
) 

cognitive_services_account = CognitiveServicesAccountKey(key=azure_ai_services_key)

skills = [split_skill, embedding_skill, entity_skill]

skillset = SearchIndexerSkillset(  
    name=skillset_name,  
    description="Skillset to chunk documents, generate embeddings, and extract location entities",  
    skills=skills,  
    index_projection=index_projections,
    cognitive_services_account=cognitive_services_account
)
  
client = SearchIndexerClient(endpoint=azure_search_service_endpoint, credential=credential)  
client.create_or_update_skillset(skillset)  
print(f"{skillset.name} created")  

ai-search-ss created


## Create an Indexer

In [6]:
from azure.search.documents.indexes.models import (
    SearchIndexer,
    IndexingSchedule
)

# Create an indexer  
indexer_name = "ai-search-idxr" 

# Schedule to run every 24 hours
schedule = IndexingSchedule(interval="P1D")

indexer_parameters = None

indexer = SearchIndexer(  
    name=indexer_name,  
    description="Indexer to index documents, generate embeddings, and extract entities",  
    skillset_name=skillset_name,  
    target_index_name=index_name,  
    data_source_name=data_source.name,
    parameters=indexer_parameters,
    schedule=schedule
)  

# Create and run the indexer  
indexer_client = SearchIndexerClient(endpoint=azure_search_service_endpoint, credential=credential)  
indexer_result = indexer_client.create_or_update_indexer(indexer)  

print(f' {indexer_name} is created and running. Give the indexer a few minutes before running a query.')  

 ai-search-idxr is created and running. Give the indexer a few minutes before running a query.


## Send a query to the search engine to check results

Executed every time a user makes a query

## Perform a Hybrid Search

In [8]:
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery
from azure.core.credentials import AzureKeyCredential

# Get credential from Azure AI Search Admin key
credential = AzureKeyCredential(azure_search_service_admin_key)
search_client = SearchClient(endpoint=azure_search_service_endpoint, 
                             credential=credential, 
                             index_name=azure_search_service_index_name)

# User Query
query = "Whats in the Arctic ocean and what is special about it?"  

# Convert query into vector form
vector_query = VectorizableTextQuery(text=query, 
                                     k_nearest_neighbors=50, 
                                     fields="text_vector",
                                     weight=1)

results = search_client.search(  
    search_text=query,  #This performs a full-text search using the query
    vector_queries= [vector_query], #This adds a vector search component
    select=["title","chunk","locations"], #Specify fields to be return in the result
    top=3
) 

for result in results:  
    print(f"Score: {result['@search.score']} \n")
    print(f"Title: {result['title']} \n")
    print(f"Chunk: {result['chunk']} \n")
    print(f"Locations: {result['locations']}")


Score: 0.03279569745063782 

Title: page-21.pdf 

Chunk: A
T

M
O

S
P

H
E

R
E

E
A

R
T

H

14

Bering Streets
Arctic Ocean

Winds from the northeast pushed sea ice southward and formed cloud streets—parallel rows of clouds—over the Bering Strait in 

January 2010. The easternmost reaches of Russia, blanketed in snow and ice, appear in the upper left. To the east, sea ice spans 

the Bering Strait. Along the southern edge of the ice, wavy tendrils of newly formed, thin sea ice predominate.

The cloud streets run in the direction of the northerly wind that helps form them. When wind blows out from a cold surface like sea 

ice over the warmer, moister air near the open ocean, cylinders of spinning air may develop. Clouds form along the upward cycle in 

the cylinders, where air is rising, and skies remain clear along the downward cycle, where air is falling. The cloud streets run toward 

the southwest in this image from the Terra satellite. 

Locations: ['Arctic Ocean', 'streets', '

## Perform a Semantic Hybrid Search

In [9]:
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery
from azure.core.credentials import AzureKeyCredential

# Get credential from Azure AI Search Admin key
credential = AzureKeyCredential(azure_search_service_admin_key)
search_client = SearchClient(endpoint=azure_search_service_endpoint, 
                             credential=credential, 
                             index_name=azure_search_service_index_name)

# User Query
query = "What can I see in the beach in the United States?"  

# Convert query into vector form
vector_query = VectorizableTextQuery(text=query, 
                                     k_nearest_neighbors=50, 
                                     fields="text_vector",
                                     weight=1)

results = search_client.search(
    query_type="semantic", 
    semantic_configuration_name='my-semantic-config',
    scoring_profile="my-scoring-profile",
    scoring_parameters=["tags-beach, 'United States'"], 
    search_text=query,
    vector_queries= [vector_query],
    select=["title","chunk","locations"],
    top=3,
)

# Sort the results by score in descending order
sorted_results = sorted(results, key=lambda x: x['@search.score'], reverse=True)

for result in sorted_results:  
    print(f"Score: {result['@search.score']} \n")
    print(f"Title: {result['title']} \n")
    print(f"Chunk: {result['chunk']} \n")
    print(f"Locations: {result['locations']}")



Score: 0.029012346640229225 

Title: page-63.pdf 

Chunk: W
a

t
e

r

56

A Lava Lamp Look at the Atlantic
Atlantic Ocean

Stretching from tropical Florida to the doorstep of Europe, the Gulf Stream carries a lot of heat, salt, and history. This river of water is 

an important part of the global ocean conveyor belt, moving water and heat from the Equator toward the far North Atlantic. It is one 

of the strongest currents on Earth and one of the most studied. Its discovery is often attributed to Benjamin Franklin, though sailors 

likely knew about the current long before they had a name for it.

This image shows a small portion of the Gulf Stream off of South Carolina as it appeared in infrared data collected by the Landsat 8 

satellite in April 2013. Colors represent the energy—heat—being emitted by the water, with cooler temperatures in purple and the 

warmest water being nearly white. Note how the Gulf Stream is not a uniform band but instead has finer streams and pockets of 



## Send query results to a language model to generate response

In [11]:
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery
from azure.core.credentials import AzureKeyCredential
from openai import AzureOpenAI

# Get credential from Azure AI Search Admin key
credential = AzureKeyCredential(azure_search_service_admin_key)
search_client = SearchClient(endpoint=azure_search_service_endpoint, 
                             credential=credential, 
                             index_name=azure_search_service_index_name)

# Azure OpenAI client
openai_client = AzureOpenAI(
    # to get version: https://learn.microsoft.com/en-us/azure/ai-services/openai/api-version-deprecation
    api_version=azure_openai_api_version,
    azure_endpoint=azure_openai_endpoint,
    api_key=azure_openai_key)

# Provide instructions to the model
SYSTEM_PROMPT="""
You are an AI assistant that helps users learn from the information found in the source material.
Answer the query using only the sources provided below.
Use bullets if the answer has multiple points.
If the answer is longer than 3 sentences, provide a summary.
Answer ONLY with the facts listed in the list of sources below. Cite your source when you answer the question
If there isn't enough information below, say you don't know.
Do not generate answers that don't use the sources below.
Query: {query}
Sources:\n{sources}
"""

# User Query
query = "Whats in the Pacific Ocean?"  

# Convert query into vector form
vector_query = VectorizableTextQuery(text=query, 
                                     k_nearest_neighbors=50, 
                                     fields="text_vector",
                                     weight=1)

results = search_client.search(
    query_type="semantic", 
    semantic_configuration_name='my-semantic-config',
    search_text=query,
    vector_queries= [vector_query],
    select=["title","chunk","locations"],
    top=5,
)

# Use a unique separator to make the sources distinct. 
# We chose repeated equal signs (=) followed by a newline because it's unlikely the source documents contain this sequence.
sources_formatted = "=================\n".join([f'TITLE: {document["title"]}, CONTENT: {document["chunk"]}, LOCATIONS: {document["locations"]}' for document in results])

response = openai_client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": SYSTEM_PROMPT.format(query=query, sources=sources_formatted)
        }
    ],
    model=azure_openai_deployment
)

print(response.choices[0].message.content)


The Pacific Ocean comprises various natural phenomena and features, including:

- **Cloud formations**: Ship tracks and stratocumulus clouds are common, with ship tracks formed by water vapor condensing around pollution particles from ships (source: page-31.pdf). Stratocumulus clouds creating a "glory" effect are also observed (source: page-17.pdf).
- **Volcanic Islands**: The Islands of the Four Mountains include Carlisle, Cleveland, Herbert, and Tana, which are peaks of volcanoes rising from the seafloor in the Aleutian Island chain (source: page-35.pdf).
- **Marine Clouds**: Along the coast of Peru, valleys are often filled with marine stratocumulus clouds that can move inland due to prevailing winds (source: page-15.pdf).
- **Hurricanes**: Hurricanes Madeline and Lester stirred the central Pacific as category 3 and 4 storms in August 2016, although they did not make landfall on Hawaii directly (source: page-27.pdf).
